# Module 2 — Thème 2 : CSV/Excel + Power Query (validation notebook)\n\nCe notebook génère :\n- `m2t2_clean_learning_dataset.csv`\n- `m2t2_quality_report.json`\n- `m2t2_data_dictionary.md`\n\nEt vérifie que `m2t2_powerquery.m` existe (script M collé par l’étudiant).\n\nEntrées attendues dans le même dossier :\n- `raw_events_messy.csv`\n- `raw_profiles_messy.xlsx`\n- `m2t2_powerquery.m` (à créer par l’étudiant)\n

## Cell 1 — Imports

In [ ]:
import os\nimport json\nimport pandas as pd\nfrom datetime import datetime\n

## Cell 2 — Robust CSV reader (separator + encoding)

In [ ]:
def read_csv_robust(path: str) -> pd.DataFrame:\n    attempts = [\n        {"sep": ";", "encoding": "utf-8"},\n        {"sep": ",", "encoding": "utf-8"},\n        {"sep": ";", "encoding": "latin1"},\n        {"sep": ",", "encoding": "latin1"},\n    ]\n    last_err = None\n    for a in attempts:\n        try:\n            df = pd.read_csv(path, **a)\n            # quick sanity: must have at least 3 columns\n            if df.shape[1] >= 3:\n                return df\n        except Exception as e:\n            last_err = e\n    raise last_err\n\nevents = read_csv_robust("raw_events_messy.csv")\nprint("Loaded events:", events.shape)\n

## Cell 2bis — Robust Excel reader (find header row)

In [ ]:
def read_profiles_robust(path: str) -> pd.DataFrame:\n    # First read without header to locate the row containing 'user_id'\n    raw = pd.read_excel(path, sheet_name=0, header=None)\n    header_row = None\n    for i in range(min(10, len(raw))):\n        row_vals = [str(v).strip().lower() for v in raw.iloc[i].tolist()]\n        if "user_id" in row_vals:\n            header_row = i\n            break\n    if header_row is None:\n        # fallback: assume first row is header\n        return pd.read_excel(path, sheet_name=0)\n    return pd.read_excel(path, sheet_name=0, header=header_row)\n\nprofiles = read_profiles_robust("raw_profiles_messy.xlsx")\nprint("Loaded profiles:", profiles.shape)\n

## Cell 3 — Standardize column names

In [ ]:
events.columns = [str(c).strip().lower() for c in events.columns]\nprofiles.columns = [str(c).strip().lower() for c in profiles.columns]\n\n# tolerant aliasing if needed\naliases = {\n    "userid": "user_id",\n    "user": "user_id",\n    "time": "event_time",\n    "timestamp": "event_time",\n    "event": "event_type"\n}\nfor k, v in aliases.items():\n    if k in events.columns and v not in events.columns:\n        events.rename(columns={k: v}, inplace=True)\n\nprint("Events columns:", list(events.columns))\nprint("Profiles columns:", list(profiles.columns))\n

## Cell 4 — Cleaning helpers

In [ ]:
def clean_str(s: pd.Series) -> pd.Series:\n    s = s.astype(str)\n    s = s.str.replace("\u00A0", " ", regex=False)  # non-breaking space\n    s = s.str.strip()\n    s = s.str.replace(r"\s+", " ", regex=True)\n    # pandas sometimes turns NaN into "nan" string after astype(str)\n    s = s.replace("nan", "")\n    return s\n\ndef normalize_country(s: pd.Series) -> pd.Series:\n    s = clean_str(s)\n    s = s.str.title()\n    # optional harmonization\n    s = s.replace({"Benin": "Bénin"})\n    return s\n\ndef normalize_channel(s: pd.Series) -> pd.Series:\n    s = clean_str(s).str.lower()\n    s = s.replace({"cellulaire": "mobile", "smartphone": "mobile"})\n    return s\n\ndef normalize_event_type(s: pd.Series) -> pd.Series:\n    s = clean_str(s).str.lower()\n    s = s.str.replace(" ", "_", regex=False)\n    s = s.str.replace("-", "_", regex=False)\n    return s\n

## Cell 5 — Ensure expected columns exist + apply cleaning

In [ ]:
for col in ["user_id", "event_time", "event_type", "theme", "country", "channel"]:\n    if col not in events.columns:\n        events[col] = ""\n\nevents["user_id"] = clean_str(events["user_id"])\nevents["event_type"] = normalize_event_type(events["event_type"])\nevents["country"] = normalize_country(events["country"])\nevents["channel"] = normalize_channel(events["channel"])\n\n# theme: coerce numeric\nevents["theme"] = pd.to_numeric(events["theme"], errors="coerce").astype("Int64")\n\n# event_time: robust parse (dayfirst common in FR)\nevents["event_time"] = pd.to_datetime(\n    events["event_time"],\n    errors="coerce",\n    dayfirst=True,\n    utc=True\n)\n\nevents.head(10)\n

## Cell 6 — Enrich from profiles (optional)

In [ ]:
if "user_id" in profiles.columns:\n    profiles["user_id"] = clean_str(profiles["user_id"])\n    if "country" in profiles.columns:\n        profiles["country"] = normalize_country(profiles["country"])\n    if "channel" in profiles.columns:\n        profiles["channel"] = normalize_channel(profiles["channel"])\n\n    keep_cols = [c for c in ["user_id", "country", "channel"] if c in profiles.columns]\n    prof_u = profiles[keep_cols].drop_duplicates("user_id")\n\n    events = events.merge(prof_u, on="user_id", how="left", suffixes=("", "_profile"))\n\n    if "country_profile" in events.columns:\n        events["country"] = events["country"].mask(\n            events["country"].eq("") | events["country"].isna(),\n            events["country_profile"]\n        )\n    if "channel_profile" in events.columns:\n        events["channel"] = events["channel"].mask(\n            events["channel"].eq("") | events["channel"].isna(),\n            events["channel_profile"]\n        )\n\n    events.drop(columns=[c for c in ["country_profile", "channel_profile"] if c in events.columns], inplace=True)\n\nprint("After enrich:", events.shape)\n

## Cell 7 — Quality checks

In [ ]:
expected_cols = ["user_id", "event_time", "event_type", "theme", "country", "channel"]\n\nreport = {\n    "created_at": datetime.utcnow().isoformat() + "Z",\n    "checks": {}\n}\n\nreport["checks"]["rows_after_load"] = int(len(events))\nreport["checks"]["has_expected_columns"] = bool(all(c in events.columns for c in expected_cols))\n\n# missing\nfor c in expected_cols:\n    report["checks"][f"missing_{c}"] = int(events[c].isna().sum()) if c in events.columns else None\n\n# duplicates (full row duplicates)\nreport["checks"]["duplicate_rows"] = int(events[expected_cols].duplicated().sum())\n\n# date range\ndt_min = events["event_time"].min()\ndt_max = events["event_time"].max()\nreport["checks"]["event_time_min"] = None if pd.isna(dt_min) else dt_min.isoformat()\nreport["checks"]["event_time_max"] = None if pd.isna(dt_max) else dt_max.isoformat()\n\n# distribution quick\nreport["checks"]["event_type_top10"] = (\n    events["event_type"].value_counts(dropna=False).head(10).to_dict()\n)\n\nreport\n

## Cell 8 — Build clean dataset (exact schema)

In [ ]:
clean = events[expected_cols].copy()\n\n# minimal integrity\nclean = clean.dropna(subset=["user_id", "event_time", "event_type"])\nclean = clean[clean["user_id"].astype(str).str.len() > 0]\n\n# enforce dtypes where possible\nclean["user_id"] = clean["user_id"].astype(str)\nclean["event_type"] = clean["event_type"].astype(str)\nclean["country"] = clean["country"].astype(str)\nclean["channel"] = clean["channel"].astype(str)\n\nclean.to_csv("m2t2_clean_learning_dataset.csv", index=False)\nprint("✅ Exported m2t2_clean_learning_dataset.csv", clean.shape)\n

## Cell 9 — Export quality report JSON

In [ ]:
with open("m2t2_quality_report.json", "w", encoding="utf-8") as f:\n    json.dump(report, f, ensure_ascii=False, indent=2)\n\nprint("✅ Exported m2t2_quality_report.json")\n

## Cell 10 — Data dictionary markdown

In [ ]:
dict_lines = []\ndict_lines.append("# Data Dictionary — Module 2 / Theme 2\\n\\n")\ndict_lines.append("## Schéma final (m2t2_clean_learning_dataset.csv)\\n")\ndict_lines.append("- **user_id**: identifiant apprenant (string)\\n")\ndict_lines.append("- **event_time**: date/heure événement (UTC, ISO)\\n")\ndict_lines.append("- **event_type**: type normalisé (ex: enrolled, opened_theme, opened_notebook, submitted, validated)\\n")\ndict_lines.append("- **theme**: numéro thème (int)\\n")\ndict_lines.append("- **country**: pays standardisé (Title Case)\\n")\ndict_lines.append("- **channel**: canal standardisé (lowercase)\\n\\n")\ndict_lines.append("## Notes qualité (extrait)\\n")\ndict_lines.append(f"- rows_after_load: {report['checks'].get('rows_after_load')}\\n")\ndict_lines.append(f"- duplicate_rows: {report['checks'].get('duplicate_rows')}\\n")\ndict_lines.append(f"- event_time_min: {report['checks'].get('event_time_min')}\\n")\ndict_lines.append(f"- event_time_max: {report['checks'].get('event_time_max')}\\n")\n\nwith open("m2t2_data_dictionary.md", "w", encoding="utf-8") as f:\n    f.writelines(dict_lines)\n\nprint("✅ Exported m2t2_data_dictionary.md")\n

## Cell 11 — Power Query M script presence check (student must paste it)

In [ ]:
m_path = "m2t2_powerquery.m"\nif not os.path.exists(m_path):\n    print("⚠️ m2t2_powerquery.m absent. Crée le fichier et colle ton script M depuis Power Query (Advanced Editor).")\nelse:\n    content = open(m_path, "r", encoding="utf-8", errors="ignore").read().strip()\n    if len(content) < 30:\n        print("⚠️ m2t2_powerquery.m présent mais trop court/vide. Colle le script M complet.")\n    else:\n        print("✅ m2t2_powerquery.m OK (non vide).")\n